In [80]:
import logging
import numpy as np
from pytplot import (
    time_string,
    time_double,
)
from pyspedas.projects.noaa.config import CONFIG
from pyspedas.utilities.dailynames import dailynames
from pyspedas.utilities.download import download


def kp_return_fraction(value):
    value = np.array(value, dtype=np.float64)
    kp_lhs = value // 10
    kp_rhs_times_3 = value % 10
    kp_rhs = kp_rhs_times_3 // 3.0
    return kp_lhs + kp_rhs / 3.0


def convert_to_float_or_int(a, outtype="int"):
    ans = []
    for v in a:
        try:
            if outtype == "float":
                ans.append(float(v))
            else:  # outtype == "int"
                ans.append(int(v))
        except ValueError:
            ans.append(0)
    return ans


def load_kp_to_df(
    trange,
    files=[],
    datatype=[],
):
    vars = []

    if files is None or len(files) == 0:
        logging.error("No files specified")
        return vars
    elif not isinstance(files, list):
        files = [files]

    if datatype is None or datatype == [] or len(datatype) == 0:
        datatype = [
            "Kp",
        ]
    elif not isinstance(datatype, list):
        datatype = [datatype]
    datatype = [d.lower() for d in datatype]

    # Each line in the files contains data for one day.
    # The first 3 quantities are measured every 3 hours (8 measurements per day).
    kpdata = []
    kptimes = []
    daytimes = []

    for kpfile in files:
        # Example line contained in these files:
        # 1701012502 73337272323302017210 18 22 12  9  9 15  7  6 120.73---070.10
        # Line length is 73 for NOAA files and 63 for WDC files (both counts include \n).

        with open(kpfile, "r") as f:
            for line in f:
                if len(line) < 63 or line.startswith("#") or line.startswith(" "):
                    # Skip lines that are less than 63 characters long (62 data and \n)
                    continue

                # Get datetimes (0:6 characters)
                year = line[0:2]
                if "00" <= year <= "69":
                    year = "20" + year
                else:
                    year = "19" + year
                month = line[2:4]
                day = line[4:6]
                ymd = year + "-" + month + "-" + day
                daytimes.append(time_double(ymd))
                for k in range(8):
                    dd = time_double(ymd + " " + "{:02d}".format(k * 3) + ":00:00")
                    kptimes.append(dd)
                for k in range(8):
                    kpdata.append(line[12 + 2 * k : 14 + 2 * k])

    # Check for empty data set. If empty, return.
    if len(kptimes) == 0:
        logging.error("No data found.")
        return vars
    if (
        len(trange) == 2
        and trange[0] != 0
        and time_double(trange[1]) > time_double(trange[0])
        and (
            time_double(trange[1]) < kptimes[0] or time_double(trange[0]) > kptimes[-1]
        )
    ):
        logging.error("No data found in time range.")
        return vars
    
    vars = pd.DataFrame({'DateTime': kptimes, 'KP': kp_return_fraction(kpdata)})
    vars['DateTime'] = pd.to_datetime(vars['DateTime'], unit='s')
    
    return vars


def noaa_load_kp(
    trange
):
    vars = []

    if len(trange) == 2:
        trangestr = time_string(time_double(trange))
        start_year = int(trangestr[0][0:4])
        end_year = int(trangestr[1][0:4])
    else:
        logging.error("Invalid time range")
        return
    if end_year > start_year + 3:  # Limit to 4 years
        end_year = start_year + 3
        trange[1] = str(end_year) + "-12-31/00:00:00"
        logging.warning(
            "Time limit is 4 years, new time range is " + trange[0] + " to " + trange[1]
        )
    elif end_year < start_year:
        end_year = start_year
        trange[1] = str(end_year) + "-12-31/00:00:00"
        logging.warning(
            "End time is before start time, new time range is "
            + trange[0]
            + " to "
            + trange[1]
        )

    # Remote site and directory
    
    remote_data_dir = CONFIG['gfz_remote_data_dir']
    pathformat = "Kp_def%Y.wdc"
    
    logging.info("Loading geomagnetic index data from " + remote_data_dir)
    remote_names = dailynames(file_format=pathformat, trange=trange)
    files = download(
        remote_file=remote_names,
        remote_path=remote_data_dir,
    )

    if len(files) == 0:
        logging.error("No files found.")
        return vars

    vars = load_kp_to_df(
        trange=trange,
        files=list(set(files)),
    )

    return vars

In [81]:
import matplotlib.pyplot as plt
df = noaa_load_kp(trange=["1991-01-01/00:00:00", "1995-01-01/23:59:59"])

df['DateTime'] = pd.to_datetime(df['DateTime'])
df['Date'] = df['DateTime'].dt.date
print(df.head(10))
# plt.figure(figsize=(10, 6))
# plt.plot(df['DateTime'], df['KP'])
# plt.title('Plot of Y vs X')
# plt.xlabel('Date Time')
# plt.ylabel('Kp Value')
# plt.grid(True)
# plt.show()


17-May-25 22:00:39: Time limit is 4 years, new time range is 1991-01-01/00:00:00 to 1994-12-31/00:00:00
17-May-25 22:00:39: Loading geomagnetic index data from https://datapub.gfz-potsdam.de/download/10.5880.Kp.0001/Kp_definitive/
17-May-25 22:00:39: File is current: /Users/soumya/Desktop/cassini_hack/Kp_def1991.wdc
17-May-25 22:00:39: File is current: /Users/soumya/Desktop/cassini_hack/Kp_def1992.wdc
17-May-25 22:00:39: File is current: /Users/soumya/Desktop/cassini_hack/Kp_def1993.wdc
17-May-25 22:00:39: File is current: /Users/soumya/Desktop/cassini_hack/Kp_def1994.wdc


             DateTime        KP        Date
0 1991-01-01 00:00:00  4.000000  1991-01-01
1 1991-01-01 03:00:00  2.666667  1991-01-01
2 1991-01-01 06:00:00  1.333333  1991-01-01
3 1991-01-01 09:00:00  1.000000  1991-01-01
4 1991-01-01 12:00:00  1.000000  1991-01-01
5 1991-01-01 15:00:00  1.000000  1991-01-01
6 1991-01-01 18:00:00  0.333333  1991-01-01
7 1991-01-01 21:00:00  0.333333  1991-01-01
8 1991-01-02 00:00:00  2.666667  1991-01-02
9 1991-01-02 03:00:00  3.666667  1991-01-02


In [82]:
df = df.groupby('Date')['KP'].mean().reset_index()
print(df)

            Date        KP
0     1991-01-01  1.458333
1     1991-01-02  2.416667
2     1991-01-03  2.333333
3     1991-01-04  1.916667
4     1991-01-05  1.958333
...          ...       ...
1456  1994-12-27  2.583333
1457  1994-12-28  2.083333
1458  1994-12-29  2.708333
1459  1994-12-30  1.750000
1460  1994-12-31  1.250000

[1461 rows x 2 columns]


In [83]:
df.info()
df.head(20)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1461 entries, 0 to 1460
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    1461 non-null   object 
 1   KP      1461 non-null   float64
dtypes: float64(1), object(1)
memory usage: 23.0+ KB


,Date,KP
0,1991-01-01,1.458333
1,1991-01-02,2.416667
2,1991-01-03,2.333333
3,1991-01-04,1.916667
4,1991-01-05,1.958333
5,1991-01-06,0.833333
6,1991-01-07,0.625000
7,1991-01-08,2.125000
8,1991-01-09,1.916667
9,1991-01-10,2.125000


In [84]:
istd = pd.read_csv('decoded_stroke_data.csv')

istd.rename(columns={'Decoded_Date': 'Date'}, inplace=True)

merged = pd.merge(df, istd, on='Date', how='outer')

istd.head()


17-May-25 22:00:39: /var/folders/fc/vy5m47nj62b3nk4p56wsjwr40000gn/T/ipykernel_20876/151638730.py:1: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  istd = pd.read_csv('decoded_stroke_data.csv')



,HOSPNUM,RDELAY,RCONSC,SEX,AGE,RSLEEP,RATRIAL,RCT,RVISINF,RHEP24,...,H14,ISC14,NK14,STRK14,HTI14,PE14,DVT14,TRAN14,NCB14,Date
0,1,17,D,M,69,Y,NaN,Y,Y,NaN,...,0,0,0,0,0,0,0,0,0,1991-01-04
1,1,10,F,M,76,Y,NaN,Y,N,NaN,...,0,0,0,0,0,0,0,0,0,1991-01-07
2,1,43,F,F,71,N,NaN,Y,N,NaN,...,0,0,0,0,0,0,0,0,0,1991-01-03
3,1,6,F,M,81,N,NaN,N,N,NaN,...,0,0,0,0,0,0,0,0,0,1991-01-07
4,4,20,F,M,78,N,NaN,N,N,NaN,...,0,0,0,0,0,0,0,0,0,1991-02-06


In [85]:
merged

,Date,KP,HOSPNUM,RDELAY,RCONSC,SEX,AGE,RSLEEP,RATRIAL,RCT,...,DEAD8,H14,ISC14,NK14,STRK14,HTI14,PE14,DVT14,TRAN14,NCB14
0,1991-01-01,1.458333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1991-01-02,2.416667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1991-01-03,2.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1991-01-04,1.916667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1991-01-05,1.958333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20891,1991-12-07,NaN,113.0,45.0,F,F,74.0,N,N,Y,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20892,1991-12-07,NaN,119.0,15.0,F,F,73.0,N,N,N,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20893,1991-12-07,NaN,189.0,8.0,D,M,84.0,Y,N,Y,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20894,1991-12-07,NaN,74.0,8.0,F,F,61.0,N,N,N,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
